In [2]:
import numpy as np
import scipy as sp
import os 
import sys
import importlib

In [3]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_steven_free_utils as msfreeu
import my_data_process_utils as mdpu
import my_meal_detection_utils as mmdu
#importlib.reload(mdu)

In [4]:
annots = mfileu.read_file('data', 'free_data_steven_annots.pkl')
#lab_data = mfileu.read_file('data', 'lab_data_steven.pkl')
free_data = mfileu.read_file('data', 'free_data_steven_right.pkl')

In [5]:
all_proba_bite = []
for subj in range(len(annots)):
    subj_proba_bite = []    
    for sess in range(len(annots[subj])):
        r = mfileu.read_file('our_test_proba', 'bite_free_'+str(subj)+'_'+str(sess)+".pkl")
        subj_proba_bite.append(r)        
        
    all_proba_bite.append(subj_proba_bite)
    

In [6]:
def find_events(proba, indices, proba_th=0.5):
    count = len(proba)
    
    res =[]
    inside = False
    for i in range(count):
        if proba[i]>=proba_th and inside==False:
            inside = True
            si = i
        elif proba[i]<proba_th and inside==True:                        
            res.append([indices[si], indices[i-1], np.amax(proba[si:i])])
            inside = False
        
    res = np.array(res)
    return res

In [27]:
def cluster_bites(bites, minPts, eps):    
    core_pt = (bites[:, -1]>=0.9)
    
    sindices = bites[:,0].astype(int)
    eindices = bites[:,1].astype(int)
    
    clusters = []    
    si, last_cpi  = -1, -1
    for i in range(0, len(bites)):        
        if last_cpi < 0:            
            if core_pt[i]:
                last_cpi = i
                si = i                
                while si>=0 and sindices[i] - eindices[si]<=eps:
                    si -= 1
            continue            
            
        if sindices[i] - eindices[last_cpi] > eps:
            if si>=0 and ( (not core_pt[i]) or (sindices[i] - eindices[i-1] > eps) ):
                clusters.append([si, i-1])                
                si = -1
                
            
            if core_pt[i] and si<0:                
                si = i
                while si>=0 and sindices[i] - eindices[si]<=eps:
                    si -= 1
        
        if core_pt[i]:
            last_cpi = i    
    
    if si>=0:
        clusters.append([si, len(bites)-1])                
    
    clusters = np.array(clusters).astype(int)    
    res = np.zeros((len(clusters), 4)).astype(int)    
    for i in range(clusters.shape[0]):        
        res[i, 0] = bites[clusters[i, 0], 0]
        res[i, 1] = bites[clusters[i, 1], 1]
        res[i, 2] = clusters[i, 1] - clusters[i, 0]+1
        res[i, 3] = np.sum(core_pt[clusters[i, 0]:clusters[i, 1]+1])
        assert res[i, 3]>=1
    
    res = res[res[:, -1]>=minPts, :2]
    return res

In [28]:
import matplotlib.pyplot as plt
%matplotlib inline

def get_result(subj, sess, minPts, eps, plot=False):
    results = []
    
    #print("Subj, sess: ", subj, sess)

    a = annots[subj][sess]        
    a = msfreeu.process_anntos(len(free_data[subj][sess]), a)
    
    proba = all_proba_bite[subj][sess]
    indices = proba[:, 0] + 40
    proba = proba[:, -1]
    #print(proba.shape, np.sum(proba))
    bite_indices = find_events(proba, indices)
    
    clusters = cluster_bites(bite_indices, minPts=minPts, eps = eps)    
    result = mmdu.get_meal_detection_results(a, clusters=clusters)
    
    ###############################################

    if not plot:
        return result

    idf = 16*60*60
    fig = plt.figure(figsize=(20,8))
    ax = plt.subplot(111)        
    ax.scatter(bite_indices[:,0]/idf, np.zeros((len(bite_indices)),)+1, marker='x', s=10, color='blue')                
    
    clrs = ['', 'blue', 'green', 'red', 'black']
    for i in range(len(a)):
        si = a[i, 0]/idf
        ei = a[i, 1]/idf
        mt = a[i, 2]            
        ax.plot([si, ei], [0.5, 0.5], color=clrs[mt], linewidth=5)

    for i in range(len(clusters)):
        si = clusters[i, 0]/idf
        ei = clusters[i, 1]/idf            
        ax.plot([si, ei], [0.75, 0.75], color='black', linewidth=5)

    plt.title(str(subj)+", "+str(sess))
    plt.xlabel('Time (Hour)', fontsize=20)        
    plt.ylim([0, 2])
    plt.grid(True)
    plt.show()        

    return result

In [29]:
def get_metric_results(minPts, eps):
    gts, acovs, clcovs = [], [], []
    for subj in range(len(annots)):
        for sess in range(len(annots[subj])):
            
            gt, ac, clc = get_result(subj, sess, minPts=minPts, eps=eps, plot=False)
            assert len(gt) == len(ac)

            gt = mdpu.add_subj_sess_to_array(gt, subj, sess, at_begin=False)
            ac = mdpu.add_subj_sess_to_array(ac, subj, sess, at_begin=False)
            clc = mdpu.add_subj_sess_to_array(clc, subj, sess, at_begin=False)

            gts = gt if len(gts)==0 else np.concatenate((gts, gt))
            acovs = ac if len(acovs)==0 else np.concatenate((acovs, ac))
            clcovs = clc if len(clcovs)==0 else np.concatenate((clcovs, clc))
            
    res = mmdu.get_metric_results(gts, acovs, clcovs)
    return res

In [30]:
res = get_metric_results(minPts=3, eps=16*60*3)
for key, val in res.items():
    print(key, " : ", val)

total  :  82
total_meal  :  49
total_snack  :  33
tp  :  61
tp_meal  :  42
tp_snack  :  19
recall  :  0.7439024390243902
recall_meal  :  0.8571428571428571
recall_snack  :  0.5757575757575758
fp  :  99
precision  :  0.38125
f1  :  0.5041322314049587
start_error  :  1053.1791158536585
start_error_meal  :  1258.704081632653
start_error_snack  :  748.0056818181819
end_error  :  602.2446646341464
end_error_meal  :  519.5803571428571
end_error_snack  :  724.9886363636364


In [1]:
for minPts in range(1, 5):
    for eps in range(10, 301, 10):    
        res = get_metric_results(minPts=minPts, eps=eps)
        
mfileu.write_file('final_results', 'ours.pkl', res)

IndentationError: expected an indented block (<ipython-input-1-305b011de8ec>, line 3)